In [1]:
from gliner import GLiNER

# Initialize GLiNER with the base model
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/781M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/781M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/home/gerald/miniconda3/envs/video-ingestion/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
def read_file_to_string(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        return content
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
        

In [13]:
import re
def split_into_sentences(text):
    # Regular expression pattern to match sentence-ending punctuation followed by a space or end of string
    sentence_endings = re.compile(r'(?<=[.!?]) +|(?<=[.!?])$')
    
    # Split the text into sentences using the compiled pattern
    sentences = sentence_endings.split(text)
    
    # Strip any leading/trailing whitespace from each sentence
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    
    return sentences

In [14]:
transcript_sentences = split_into_sentences(transcript)

In [5]:
transcript = read_file_to_string('/mnt/data3/AI/software/VideoRAG/Lexington/DPa2iRgzadM.raw_transcript')


In [7]:
labels = ["Person", "Organizations", "Date", "Positions", "Locations"]

In [18]:
# Perform entity prediction
entities = model.batch_predict_entities(transcript_sentences, labels, threshold=0.5)

In [23]:
def group_by_label(data):
    # Initialize an empty dictionary to hold the results
    result = {}

    # Iterate through each item in the list
    for sentence in data:
        for entry in sentence:
            # Check if the entry is a dictionary (to skip empty lists)
            if isinstance(entry, dict):
                label = entry['label']
            
                # Create a new dictionary with only 'text' and 'score'
                text_score_dict = {'text': entry['text'], 'score': entry['score']}
            
                # Add to the result dictionary under the appropriate label
                if label not in result:
                    result[label] = []
                result[label].append(text_score_dict)

    return result

In [24]:
entities_by_label = group_by_label(entities)

In [68]:
def merge_similar_texts(data):
    # Initialize an empty dictionary to hold the results
    result = {}

    # Iterate through each label in the input data
    for label, entries in data.items():
        # Use a dictionary to store unique texts with their maximum scores
        unique_entries = {}

        # Process each entry in the list of entries for the current label
        for entry in entries:
            text = entry['text']
            score = entry['score']

            if text in ['he','she', 'I', 'me', 'her','him','they', 'we']:
                continue
            # If the text is already in the unique_entries, update the score if it's higher
            if text in unique_entries:
                unique_entries[text] = max(unique_entries[text], score)
            else:
                unique_entries[text] = score
                
        #print(unique_entries)
        # Convert the unique_entries dictionary back to a list of dictionaries
        result[label] = [{'text': text, 'score': score} for text, score in unique_entries.items()]   
    return result


In [69]:
entities_merged = merge_similar_texts(entities_by_label)